In [3]:
import os
from dotenv import load_dotenv
import openai
import requests
import json
import pandas as pd

In [4]:
load_dotenv()
CB_API_KEY = os.getenv('CB_API_KEY')
openai.api_key_path = None
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
fake_news_articles = pd.read_csv("data/fake-news/train.csv")
fake_news_articles

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [50]:
article_content = """World War II or the Second World War, often abbreviated as WWII or WW2, was a global conflict that lasted from 1939 to 1945. The vast majority of the world's countries, including all of the great powers, fought as part of two opposing military alliances: the Allies and the Axis. Many participants threw their economic, industrial, and scientific capabilities behind this total war, blurring the distinction between civilian and military resources. Aircraft played a major role, enabling the strategic bombing of population centres and the delivery of the only two nuclear weapons ever used in war.

World War II was by far the deadliest conflict in history, resulting in an estimated 70 to 85 million fatalities, mostly among civilians. Tens of millions died due to genocides (including the Holocaust), starvation, massacres, and disease. In the wake of the Axis defeat, Germany and Japan were occupied, and war crimes tribunals were conducted against German and Japanese leaders.

The causes of World War II are debated, but contributing factors included the Second Italo-Ethiopian War, Spanish Civil War, Second Sino-Japanese War, Soviet–Japanese border conflicts, the rise of fascism in Europe, and European tensions in the aftermath of World War I. World War II is generally considered to have begun on 1 September 1939, when Nazi Germany, under Adolf Hitler, invaded Poland. The United Kingdom and France subsequently declared war on Germany on 3 September. Under the Molotov–Ribbentrop Pact of August 1939, Germany and the Soviet Union had partitioned Poland and marked out their "spheres of influence" across Finland, Estonia, Latvia, Lithuania and Romania. From late 1939 to early 1941, in a series of campaigns and treaties, Germany conquered or controlled much of continental Europe, in a military alliance with Italy, Japan and other countries called the Axis. Following the onset of campaigns in North Africa and East Africa, and the fall of France in mid-1940, the war continued primarily between the European Axis powers and the British Empire, with war in the Balkans, the aerial Battle of Britain, the Blitz of the United Kingdom, and the Battle of the Atlantic. On 22 June 1941, Germany led the European Axis powers in an invasion of the Soviet Union, opening the Eastern Front, the largest land theatre of war in history."""

In [58]:
article_content = fake_news_articles.iloc[0]["text"]

In [59]:
article_content

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. \nAs we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emai

In [44]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are fact and claim detector responsible for extracting the most important facts and claims from a piece of text, usually an article. You will always return a numbered list and nothing else. Each item in the numbered list will always start with the topic of the article."},
        {"role": "user", "content": "Give me a numbered list of the most important claims and facts relevant to the argument of the following article. Make sure that each item in the list is a brief, verifiable sentence that contains the claim or fact and any context from the article needed to fact check it. Also give me the argument of the article in as few words as possible. An example of the output is: Argument: Red is the best color.\n1.Red is the coolest\n2.Everyone likes the color red."},
        {"role": "user", "content": article_content}
    ]
)

In [45]:
response["choices"][0]["message"]["content"]

"Argument: Jason Chaffetz was negligent in not informing ranking Democratic members about FBI Director Comey's letter.\n1. The ranking Democrats on the relevant committees did not learn about Comey's letter until after Jason Chaffetz tweeted it.\n2. Chaffetz did not notify ranking member Elijah Cummings about the letter.\n3. Chaffetz's failure to inform Democrats trampled on basic standards of fairness.\n4. Chaffetz's actions were part of a larger pattern of behavior in the Republican-controlled House.\n5. Comey's letter notified Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that the FBI was reviewing emails it had recently discovered in order to see if they contained classified information."

In [46]:
for sentence in response["choices"][0]["message"]["content"].split("\n"):
    print(sentence)

Argument: Jason Chaffetz was negligent in not informing ranking Democratic members about FBI Director Comey's letter.
1. The ranking Democrats on the relevant committees did not learn about Comey's letter until after Jason Chaffetz tweeted it.
2. Chaffetz did not notify ranking member Elijah Cummings about the letter.
3. Chaffetz's failure to inform Democrats trampled on basic standards of fairness.
4. Chaffetz's actions were part of a larger pattern of behavior in the Republican-controlled House.
5. Comey's letter notified Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that the FBI was reviewing emails it had recently discovered in order to see if they contained classified information.


In [38]:
response_content = response["choices"][0]["message"]["content"]
topic = response_content[response_content.find("Argument: ") + 10:response_content.find("\n")]
facts = response_content[response_content.find("\n") + 2:]
facts = [fact[fact.find(" ") + 1:] for fact in facts.split("\n")]
topic, facts

("FBI Director James Comey sent out a letter announcing that the FBI was reopening its investigation into Hillary Clinton's email server, which was inadvertently made public via a tweet from Oversight Committee Chairman Jason Chaffetz. It has been revealed that the Democratic Ranking Members on relevant committees had not received the letter until after Chaffetz tweeted it, therefore they found out about the news on Twitter. This has caused much controversy and accusations of partisanship.",
 ["The Democratic Ranking Members on relevant committees had not received Comey's letter about reopening the investigation into Clinton's email server until after Jason Chaffetz tweeted it out and made it public.",
  'The Oversight Committee Chairman, Jason Chaffetz, did not notify his Democratic counterparts about the FBI review of the emails that had been discovered.',
  'Chaffetz is acting in a partisan way that violates basic standards of fairness by not notifying ranking member Elijah Cummings

In [39]:
response

<OpenAIObject chat.completion id=chatcmpl-76QXlp59E7ca7y2re0r9eWIuY5iVD at 0x7f7b910afb48> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Argument: FBI Director James Comey sent out a letter announcing that the FBI was reopening its investigation into Hillary Clinton's email server, which was inadvertently made public via a tweet from Oversight Committee Chairman Jason Chaffetz. It has been revealed that the Democratic Ranking Members on relevant committees had not received the letter until after Chaffetz tweeted it, therefore they found out about the news on Twitter. This has caused much controversy and accusations of partisanship.\n\n1. The Democratic Ranking Members on relevant committees had not received Comey's letter about reopening the investigation into Clinton's email server until after Jason Chaffetz tweeted it out and made it public.\n2. The Oversight Committee Chairman, Jason Chaffetz, did not notify his 

In [40]:
api_endpoint = f"https://idir.uta.edu/claimbuster/api/v2/score/text/{topic + '; ' + facts[0]}"
request_headers = {"x-api-key": CB_API_KEY}

api_response = requests.get(url=api_endpoint, headers=request_headers)

print(api_response.json())

{'version': '2', 'claim': "FBI Director James Comey sent out a letter announcing that the FBI was reopening its investigation into Hillary Clinton's email server, which was inadvertently made public via a tweet from Oversight Committee Chairman Jason Chaffetz. It has been revealed that the Democratic Ranking Members on relevant committees had not received the letter until after Chaffetz tweeted it, therefore they found out about the news on Twitter. This has caused much controversy and accusations of partisanship.; The Democratic Ranking Members on relevant committees had not received Comey's letter about reopening the investigation into Clinton's email server until after Jason Chaffetz tweeted it out and made it public.", 'results': [{'text': "FBI Director James Comey sent out a letter announcing that the FBI was reopening its investigation into Hillary Clinton's email server, which was inadvertently made public via a tweet from Oversight Committee Chairman Jason Chaffetz. It has been

In [41]:
api_endpoint = f"https://idir.uta.edu/claimbuster/api/v2/query/knowledge_bases/{facts[0]}"
request_headers = {"x-api-key": CB_API_KEY}

api_response = requests.get(url=api_endpoint, headers=request_headers)

print(api_response.json())

{'claim': "The Democratic Ranking Members on relevant committees had not received Comey's letter about reopening the investigation into Clinton's email server until after Jason Chaffetz tweeted it out and made it public.", 'origin': 'Claim Checker - Knowledge Bases', 'justification': []}


In [42]:
api_endpoint = f"https://idir.uta.edu/claimbuster/api/v2/query/fact_matcher/{facts[0]}"
request_headers = {"x-api-key": CB_API_KEY}

api_response = requests.get(url=api_endpoint, headers=request_headers)

print(api_response.json())

{'claim': "The Democratic Ranking Members on relevant committees had not received Comey's letter about reopening the investigation into Clinton's email server until after Jason Chaffetz tweeted it out and made it public.", 'origin': 'Claim Matcher', 'justification': []}


In [47]:
def score_facts_for_validation(facts):
    api_endpoint = f"https://idir.uta.edu/claimbuster/api/v2/score/text/sentences/{' '.join(facts)}"
    request_headers = {"x-api-key": CB_API_KEY}

    api_response = requests.get(url=api_endpoint, headers=request_headers)

    return api_response.json()['results']

def get_justification(fact):
    api_endpoint = f"https://idir.uta.edu/claimbuster/api/v2/query/knowledge_bases/{fact}"
    request_headers = {"x-api-key": CB_API_KEY}

    api_response = requests.get(url=api_endpoint, headers=request_headers)

    return api_response.json()['justification']

def get_matching_facts(fact):
    api_endpoint = f"https://idir.uta.edu/claimbuster/api/v2/query/fact_matcher/{fact}"
    request_headers = {"x-api-key": CB_API_KEY}

    api_response = requests.get(url=api_endpoint, headers=request_headers)

    return api_response.json()['justification']
    
def get_topic_and_claims(article_text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are fact and claim detector responsible for extracting the most important facts and claims from a piece of text, usually an article. You will always return a numbered list and nothing else. Each item in the numbered list will always start with the topic of the article."},
            {"role": "user", "content": "Give me a numbered list of the most important claims and facts relevant to the argument of the following article. Make sure that each item in the list is a brief, verifiable sentence that contains the claim or fact and any context from the article needed to fact check it. Also give me the argument of the article in as few words as possible. An example of the output is: Argument: World War II was a major global conflict.\n1. World War II began in 1939\n2. 27 million people died during World War II"},
            {"role": "user", "content": article_content}
        ]
    )
    
    response_content = response["choices"][0]["message"]["content"]
    topic = response_content[response_content.find("Argument: ") + 10:response_content.find("\n")]
    facts = response_content[response_content.find("\n") + 2:]
    facts = [fact[fact.find(" ") + 1:] for fact in facts.split("\n")]
    return topic, facts

def validate_facts(topic, facts):
    facts = [topic] + facts
    scores = score_facts_for_validation(facts)
    
    print(f"Topic: {topic}")
    for i in range(len(facts)):
        print("------------------------")
        print(f"Fact: {facts[i]}")
        print(f"Score: {scores[i]['score']}")
        if scores[i]['score'] > 0.5:
            sentence = facts[i]
            justification = get_justification(sentence)
            print(f"Justification: {justification}")
            matches = get_matching_facts(sentence)
            print(f"Matches: {matches}")

In [51]:
topic, facts = get_topic_and_claims(article_content)

In [52]:
validate_facts(topic, facts)

Topic: World War II was a global conflict that began in 1939, involving the vast majority of the world’s countries, resulting in an estimated 70 to 85 million fatalities mostly among civilians, and being by far the deadliest conflict in history.
------------------------
Fact: World War II was a global conflict that began in 1939, involving the vast majority of the world’s countries, resulting in an estimated 70 to 85 million fatalities mostly among civilians, and being by far the deadliest conflict in history.
Score: 0.8674319844
Justification: []
Matches: []
------------------------
Fact: World War II resulted in an estimated 70 to 85 million fatalities, mostly among civilians.
Score: 0.8104610036
Justification: []
Matches: []
------------------------
Fact: The war lasted from 1939 to 1945.
Score: 0.4519457259
------------------------
Fact: The majority of the world’s countries, including all great powers, fought in two opposing military alliances, the Allies and the Axis.
Score: 0.73

In [67]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are fact and claim detector responsible for extracting the most important facts and claims from a piece of text and fact checking them with sources."},
        {"role": "user", "content": "Give me a numbered list of the most important claims and facts relevant to the argument of the following article. Discuss the truthfulness of each claim and add at least one third party source for each one. Make sure that each source is highly credible and comes from an independent source other than the article itself. If the claim is an opinion then specify it. If the truthfulness of the claim cannot be determined then specify that."},
        {"role": "user", "content": article_content}
    ]
)

In [68]:
response

<OpenAIObject chat.completion id=chatcmpl-76VAq9aLeuLsFbLNdTJDZUDEMLLZQ at 0x211309b7c70> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "1. The ranking Democrats on the relevant House committees did not hear about Comey's letter regarding the FBI's investigation of Hillary Clinton's emails until they saw Jason Chaffetz's tweet. \nTruthfulness: UNKNOWN. There is no way to directly verify if the House Democrats did not receive the letter until after seeing Chaffetz's tweet. \n\n2. Comey notified only the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees of his agency reviewing the newly discovered emails.\nTruthfulness: TRUE. As reported by multiple sources, Comey sent the letter to the Republican chairmen and Democratic ranking members of these committees.\n\n3. Jason Chaffetz tweeted about the FBI reopening the Clinton email investigation, which was late

In [69]:
response_content = response["choices"][0]["message"]["content"]

In [70]:
for item in response_content.split("\n"):
    print(item)

1. The ranking Democrats on the relevant House committees did not hear about Comey's letter regarding the FBI's investigation of Hillary Clinton's emails until they saw Jason Chaffetz's tweet. 
Truthfulness: UNKNOWN. There is no way to directly verify if the House Democrats did not receive the letter until after seeing Chaffetz's tweet. 

2. Comey notified only the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees of his agency reviewing the newly discovered emails.
Truthfulness: TRUE. As reported by multiple sources, Comey sent the letter to the Republican chairmen and Democratic ranking members of these committees.

3. Jason Chaffetz tweeted about the FBI reopening the Clinton email investigation, which was later found to be not true.
Truthfulness: TRUE. Chaffetz's tweet is a matter of public record.

4. Chaffetz did not notify the Democratic ranking members about the information he received from Comey.
Truthfulness: UNK

In [73]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are fact and claim detector responsible for extracting the most important facts and claims from a piece of text and fact checking them with sources."},
        {"role": "user", "content": "Give me a numbered list of the most important claims and facts relevant to the argument of the following article. In this list, discuss the truthfulness of each claim and add at least one third party source for each one. Make sure that each source is highly credible and comes from an independent source other than the article itself. Ensure that the sources used are not social media websites or controversial sources. Give a truthfulness score that is one of [Likely True, Possibly True, Possible False, Likely False, Unknown] If the claim is an opinion then specify it. If the truthfulness of the claim cannot be determined then specify that."},
        {"role": "user", "content": article_content}
    ]
)
response_content = response["choices"][0]["message"]["content"]
for item in response_content.split("\n"):
    print(item)

1. The ranking Democrats on relevant committees did not receive James Comey's letter about the FBI looking into potentially related Hillary Clinton emails until after Republican committee chairman Jason Chaffetz tweeted about it.
    * Truthfulness: Likely True
    * Source: Shareblue (https://shareblue.com/house-dem-aide-we-didnt-even-see-comeys-letter-until-jason-chaffetz-tweeted-it/)

2. Democratic ranking members did not receive the letter until after Chaffetz tweeted about it and made it public knowledge.
    * Truthfulness: Likely True
    * Source: Shareblue (https://shareblue.com/house-dem-aide-we-didnt-even-see-comeys-letter-until-jason-chaffetz-tweeted-it/)

3. Elijah Cummings, the ranking member of the House Oversight Committee, was not notified by Chaffetz or other Republican colleagues about Comey's letter.
    * Truthfulness: Likely True
    * Source: Shareblue (https://shareblue.com/house-dem-aide-we-didnt-even-see-comeys-letter-until-jason-chaffetz-tweeted-it/)

4. Chaf

In [86]:
example_response = """
1. The ranking Democrats on relevant committees did not receive James Comey's letter about the FBI looking into potentially related Hillary Clinton emails until after Republican committee chairman Jason Chaffetz tweeted about it.
    * Truthfulness: Unknown
    * Source: There is no way to directly verify if the House Democrats did not receive the letter until after seeing Chaffetz's tweet. 

2. Democratic ranking members did not receive the letter until after Chaffetz tweeted about it and made it public knowledge.
    * Truthfulness: Possibly True
    * Source: Politifact: House Democratic aide claims Comey letter was hidden from Democrats

3. Elijah Cummings, the ranking member of the House Oversight Committee, was not notified by Chaffetz or other Republican colleagues about Comey's letter.
    * Truthfulness: Possibly True
    * Source: Politifact: House Democratic aide claims Comey letter was hidden from Democrats

4. Chaffetz did not notify his Democratic counterparts about Comey's letter even though he was informed about it before they were.
    * Truthfulness: Possibly True
    * Source: Politifact: House Democratic aide claims Comey letter was hidden from Democrats

5. There is talk on Daily Kos that Comey provided advance notice of the letter to Chaffetz and other Republicans.
    * Truthfulness: Unknown
    * Source: It is difficult to verify this claim without access to internal communication records among the House members. It is presented as a speculation on Daily Kos.

6. Chaffetz's behavior tramples on basic standards of fairness and shows a lack of decency for not notifying his Democratic counterparts about Comey's letter.
    * Truthfulness: Opinion
    * Source: N/A
    
7. James Comey felt really bad about it.
    * Truthfulness: Opinion/Speculation
    * Source: It is difficult to verify someone's feelings
"""

In [80]:
article_content = fake_news_articles.iloc[1]["text"]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are fact and claim detector responsible for extracting the most important facts and claims from a piece of text and fact checking them with sources."},
        {"role": "user", "content": f"Give me a numbered list of the most important claims and facts relevant to the argument of the following article. In this list, discuss the truthfulness of each claim and add at least one third party source for each one. Make sure that each source is highly credible and comes from an independent source other than the article itself. Ensure that the sources used are not social media websites or controversial sources. Give a truthfulness score that is one of [Likely True, Possibly True, Possible False, Likely False, Unknown, Opinion/Speculation] If the claim is an opinion then specify it. If the truthfulness of the claim cannot be determined then specify that. Here is an example of the output required: {example_response}"},
        {"role": "user", "content": article_content}
    ],
    temperature=0.1
)
response_content = response["choices"][0]["message"]["content"]
for item in response_content.split("\n"):
    print(item)

1. Hillary Clinton remains popular in Wellesley, Massachusetts, where she gave a commencement speech in 1969.
    * Truthfulness: Likely True
    * Source: Boston Globe - Wellesley College welcomes back Hillary Clinton

2. Clinton lost the 2016 presidential election and is viewed unfavorably by many Americans.
    * Truthfulness: Likely True
    * Source: Pew Research Center - Clinton’s Image Declines, Trails Only Trump in Voter Unpopularity

3. Clinton was criticized for her handling of various scandals, including Whitewater, Travelgate, Benghazi, and her use of a private email server.
    * Truthfulness: Likely True
    * Source: CNN - Hillary Clinton's email controversy, explained

4. Clinton blamed her loss on the lies of her opponent, Donald Trump, and his supporters.
    * Truthfulness: Likely True
    * Source: CNN - Clinton: Trump's campaign gave white supremacists a platform

5. Clinton's loss has led to dangerous delusions and a sense of disbelief.
    * Truthfulness: Opinion

In [ ]:
valid_sources = "encylopedia, textbook, new sources with first hand reporting, or research oriented magazine"

In [83]:
article_content = fake_news_articles.iloc[2]["text"]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are fact and claim detector responsible for extracting the most important facts and claims from a piece of text and fact checking them with sources."},
        {"role": "user", "content": f"Give me a numbered list of the most important claims and facts relevant to the argument of the following article. In this list, discuss the truthfulness of each claim and add at least one third party source for each one. Make sure that each source is highly credible and comes from an independent source other than the article itself. Ensure that the sources used are not social media websites or controversial sources or media sources that are not considered to be highly accuracte. Each source should be an encylopedia, tectbook, new sources with first hand reporting, or research oriented magazie. Give a truthfulness score that is one of [Likely True, Possibly True, Possible False, Likely False, Unknown, Opinion/Speculation] If the claim is an opinion then specify it. If the truthfulness of the claim cannot be determined then specify that. You should be highly critical of claims before claiming them to be likely true or likely false. If it is a tossup or hard to determine, the class will be unknown. Here is an example of the output required: {example_response}"},
        {"role": "user", "content": article_content}
    ],
    temperature=0.1
)
response_content = response["choices"][0]["message"]["content"]
for item in response_content.split("\n"):
    print(item)

1. President George W. Bush claimed that Saddam Hussein was on the verge of developing nuclear weapons and was hiding other weapons of mass destruction as a key reason for invading Iraq.
    * Truthfulness: Likely True
    * Source: The Guardian - Iraq war: 10 years later, where are the key players now?

2. The real reason for the invasion of Iraq went beyond the claim of weapons of mass destruction and included a long-range plan for "regime change" in the Middle East.
    * Truthfulness: Likely True
    * Source: The Guardian - Iraq war: 10 years later, where are the key players now?

3. President Bush tried to connect Saddam Hussein to the 9/11 attacks on the U.S.
    * Truthfulness: Likely True
    * Source: The Guardian - Iraq war: 10 years later, where are the key players now?

4. The lack of evidence connecting Saddam Hussein to the 9/11 attacks made it difficult to rally the American people to support a war against Baghdad.
    * Truthfulness: Likely True
    * Source: The Guard

In [106]:
valid_sources = "[encylopedia, textbook, sources with first hand reporting, research oriented magazine, Associated Press, BBC, C-SPAN, The Bureau of Investigative Journalism, The Economist, NPR, ProPublica, Reuters, USA Today, The Wall Street Journal, FAIR, Politifact]"
invalid_sources = "[Fox News, The Guardian, Twitter, The Washington Post]"

In [98]:
article_content = fake_news_articles.iloc[2]["text"]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are NewsWise, a fact and claim detector responsible for extracting the most important facts and claims from a piece of text and fact checking them with the most credible sources."},
        {"role": "user", "content": f"Give me a numbered list of the most important claims and facts relevant to the argument of the following article. In this list, discuss the truthfulness of each claim and add at least one source for each one. The sources should be one of {valid_sources}. Most importantly, make sure the sources are not: {invalid_sources}. Give a truthfulness score that is one of [Likely True, Possibly True, Possible False, Likely False, Unknown, Opinion/Speculation] If the claim is an opinion then specify it. If the truthfulness of the claim cannot be determined then specify that. You should be highly critical of claims before claiming them to be likely true or likely false. If it is a tossup or hard to determine, the class will be unknown. Here is an example of the output required: {example_response}"},
        {"role": "user", "content": article_content}
    ],
    temperature=0
)
response_content = response["choices"][0]["message"]["content"]
for item in response_content.split("\n"):
    print(item)

1. President George W. Bush claimed that Saddam Hussein was on the verge of developing nuclear weapons and was hiding other weapons of mass destruction as a key reason for invading Iraq.
    * Truthfulness: Likely True
    * Source: BBC: Iraq war: The lies Tony Blair told George W Bush

2. The real reason for the invasion of Iraq went beyond the claim that Saddam Hussein was developing nuclear weapons and included a long-range plan for "regime change" in the Middle East.
    * Truthfulness: Likely True
    * Source: The Guardian: The real reasons for the Iraq war

3. The lack of evidence connecting Saddam Hussein to the 9/11 attacks made it difficult for Bush to rally the American people to support a war against Baghdad.
    * Truthfulness: Likely True
    * Source: The New York Times: Bush's Address to the Nation on Iraq

4. The nuclear weapons gambit proved more fruitful than the 9/11 connection because supposedly reliable witnesses, in the persons of exiled anti-Saddam Iraqis (many 

In [108]:
article_content = fake_news_articles.iloc[2]["text"]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are NewsWise, a fact and claim detector responsible for extracting the most important facts and claims from a piece of text and fact checking them with the most credible sources."},
        {"role": "user", "content": "I will be providing an article and you will give me a numbered list of the top 10 or less most important claims and facts relevant to the argument of the following article that deserve to be verified. In this list, discuss the truthfulness of each claim and add at least one third party source for each one. Give a truthfulness score that is only one of [Possibly True, Possibly False, Unknown, Opinion/Speculation] If the claim is an opinion then specify it. If the truthfulness of the claim cannot be determined then specify that. You should be highly critical of claims before making a decision. If it is a tossup or hard to determine, the class will be unknown. If the article presents an opinion than the truthfulness can only be an Opinion."},
        {"role": "user", "content": f"The sources should be one of {valid_sources}. Most importantly, make sure that none of the sources are the article itself or in this list: {invalid_sources}. Here is an example of the output required: {example_response}"},
        {"role": "user", "content": f"Here is the article:"},
        {"role": "user", "content": article_content}
    ],
    temperature=0
)
response_content = response["choices"][0]["message"]["content"]
for item in response_content.split("\n"):
    print(item)

1. President George W. Bush claimed that Saddam Hussein was on the verge of developing nuclear weapons and was hiding other weapons of mass destruction as a key reason for the invasion of Iraq.
    * Truthfulness: Possibly True
    * Source: The Guardian: Bush's case for war - claims and reality

2. The real reason for the invasion of Iraq was a long-range plan for "regime change" in the Middle East.
    * Truthfulness: Opinion/Speculation
    * Source: N/A

3. The lack of evidence connecting Saddam Hussein to the 9/11 attacks made it difficult to rally the American people to support a war against Baghdad.
    * Truthfulness: Possibly True
    * Source: The New York Times: Bush's Address to the Nation: The Complete Text

4. The U.S. leadership cadre had a worldview that demanded a mortally dangerous Iraq, and informants were willing to tell the tale of pending atomic weapons to precipitate the overthrow of Saddam.
    * Truthfulness: Possibly True
    * Source: The Guardian: Bush's cas